In [1]:
import os
import re
import argparse
from src.constant import output_dir
from src.utils.utils import save_md, save_json, load_json, extract_plot_list
from src.generation.outline_generator import generate_outline
from src.generation.chapter_reorder import reorder_chapters
from src.generation.generate_characters import generate_characters_v1
from src.generation.expand_story import expand_story_v1
from src.compile_story import compile_full_story_by_chapter
from src.enhance_story import enhance_story_with_transitions, polish_dialogues_in_story
from src.generation.dialogue_inserter import analyze_dialogue_insertions, run_dialogue_insertion
from src.utils.utils import extract_behavior_llm, convert_dialogue_dict_to_list
from src.sync.plot_sync_manager import sync_plot_and_dialogue_from_behavior
from src.sync.auto_propagate_plot_update import auto_propagate_plot_update
from src.analysis.character_state_tracker import run_character_state_tracker
from src.utils.logger import append_log, build_log_record, build_simple_log, init_log_path
from src.version_namer import build_version_name 

In [2]:
# 在notebook中运行
from src.utils.utils import load_json
story = load_json("data/output/小红帽_科幻_linear_T0.7_s42/story.json")
characters = load_json("data/output/小红帽_科幻_linear_T0.7_s42/characters.json")
dialogue_marks = load_json("data/output/小红帽_科幻_linear_T0.7_s42/dialogue_marks.json")

# 检查第一个dialogue的sentence字段
first_sentence = dialogue_marks[0]["sentence"] 
print(f"当前sentence长度: {len(first_sentence)}")
print(f"内容: {first_sentence}")

当前sentence长度: 206
内容: 清晨,玛拉身穿祖母发明的红色防护服,背着装有医疗数据的密封包裹,谨慎地穿行在被辐射侵蚀的森林边缘.她的每一步都小心翼翼,警惕着周围可能出现的变异生物.防护服内的AI助手低声提醒她注意前方的热源反应.远处,科研站的轮廓在晨雾中若隐若现,玛拉加快脚步,心中既有对祖母的思念,也有完成任务的坚定信念.突然,通讯频道中传来AI狼的干扰信号,玛拉屏住呼吸,依靠AI助手的加密防护,继续向科研站前进,准备迎接即将到来的挑战.


In [3]:
def split_plot_into_sentences(plot_text, min_length=2):
    """
    最佳实践版本：结合各种方法的优点
    """
    if not plot_text: 
        return []
    
    text = plot_text.strip()
    
    # 替换省略号
    text = text.replace("...", "〈ELLIPSIS〉")
    text = text.replace("……", "〈ELLIPSIS_CN〉")
    
    # 使用正则表达式找到所有句子
    # 贪婪匹配到标点符号为止
    pattern = r'.+?[.。!！?？;；]+'
    
    # 找到所有匹配的句子
    sentences = re.findall(pattern, text)
    
    # 检查是否有剩余的文本（没有标点结尾的）
    last_pos = 0
    for match in re.finditer(pattern, text):
        last_pos = match.end()
    
    if last_pos < len(text):
        remaining = text[last_pos:].strip()
        if remaining:
            sentences.append(remaining)
    
    # 处理结果
    result = []
    for sentence in sentences:
        # 还原省略号
        sentence = sentence.replace("〈ELLIPSIS〉", "...")
        sentence = sentence.replace("〈ELLIPSIS_CN〉", "……")
        sentence = sentence.strip()
        
        # 只过滤真正太短的（比如单个字符）
        if sentence and len(sentence) >= min_length:
            result.append(sentence)
    
    return result

In [4]:
test_plot = story[0]["plot"]
sentences = split_plot_into_sentences(test_plot)
print(f"原plot长度: {len(test_plot)}")
print(f"分割成: {len(sentences)} 个句子")
for i, s in enumerate(sentences):
    print(f"{i+1}. {s}")

原plot长度: 206
分割成: 5 个句子
1. 清晨,玛拉身穿祖母发明的红色防护服,背着装有医疗数据的密封包裹,谨慎地穿行在被辐射侵蚀的森林边缘.
2. 她的每一步都小心翼翼,警惕着周围可能出现的变异生物.
3. 防护服内的AI助手低声提醒她注意前方的热源反应.
4. 远处,科研站的轮廓在晨雾中若隐若现,玛拉加快脚步,心中既有对祖母的思念,也有完成任务的坚定信念.
5. 突然,通讯频道中传来AI狼的干扰信号,玛拉屏住呼吸,依靠AI助手的加密防护,继续向科研站前进,准备迎接即将到来的挑战.


In [ ]:
def analyze_dialogue_insertions_v2(story, characters):
    from src.utils.utils import split_plot_into_sentences, generate_response, convert_json, extract_behavior_llm
    from src.generation.dialogue_inserter import generate_dialogue_for_insertion
    chapter_results = []       # 兼容格式（6个章节）
    behavior_timeline = []     # 详细的behavior track
    sentence_results = []      # 句子级详细数据（用于分析）
    
    for chapter in story:
        chapter_id = chapter.get("chapter_id", "Unknown")
        scene = chapter.get("scene", "")
        plot = chapter.get("plot", "")
        
        # 1. 句子级分析
        sentences = split_plot_into_sentences(plot)
        print(f"章节{chapter_id}分割为{len(sentences)}个句子")
        
        # 2. 直接LLM分析句子（不调用原函数）
        msg = [{
            "role": "system", 
            "content": f"""对每句剧情判断是否需要插入对话：
句子列表：{sentences}
演员表：{characters}
格式：[{{"sentence":"...", "need_to_action":0 or 1, "actor_list":["演员A"]}}]
只返回JSON。"""
        }]
        response = generate_response(msg)
        sentence_analysis = convert_json(response)
        
        # 3. 生成对话和behavior
        chapter_dialogues = []
        all_actors = set()
        
        for sent_idx, result in enumerate(sentence_analysis):
            # 保存句子级详细数据
            sentence_result = {
                "chapter_id": chapter_id,
                "sentence_index": sent_idx,
                "sentence": result.get("sentence", ""),
                "need_to_action": result.get("need_to_action", 0),
                "actor_list": result.get("actor_list", [])
            }
            sentence_results.append(sentence_result)
            
            if result.get("need_to_action") == 1:
                # 生成对话
                dialogue = generate_dialogue_for_insertion(
                    result["sentence"], 
                    result["actor_list"],
                    [plot],
                    characters
                )
                chapter_dialogues.extend(dialogue)
                all_actors.update(result["actor_list"])
                
                # 提取behavior
                if dialogue:
                    try:
                        behavior = extract_behavior_llm(dialogue)
                        
                        for character, behaviors in behavior.items():
                            for behavior_state in behaviors:
                                behavior_timeline.append({
                                    "chapter_id": chapter_id,
                                    "sentence_index": sent_idx,
                                    "sentence": result["sentence"][:50] + "..." if len(result["sentence"]) > 50 else result["sentence"],
                                    "character": character,
                                    "behavior": behavior_state,
                                    "scene_context": scene,
                                    "dialogue_trigger": True
                                })
                    except Exception as e:
                        print(f"⚠️ Behavior提取失败: {e}")
        
        # 4. 组装章节级结果（兼容格式）
        chapter_result = {
            "sentence": plot,  # 完整章节
            "need_to_action": 1 if chapter_dialogues else 0,
            "actor_list": list(all_actors),
            "dialogue": chapter_dialogues
        }
        chapter_results.append(chapter_result)
    
    return chapter_results, sentence_results, behavior_timeline

In [12]:
# from src.utils.utils import generate_response, convert_json


# # 测试
# test_result = analyze_dialogue_insertions_v2(story[:1], characters)
# print(f"第1章总共{len(test_result)}个句子")

# # 查看每句话的对话安排
# for item in test_result:
#     chapter_id = item["chapter_id"]
#     sentence_idx = item["sentence_index"]
#     need_action = item["need_to_action"]
#     actors = item["actor_list"]
#     sentence = item["sentence"][:50] + "..."
    
#     if need_action == 1:
#         print(f"{chapter_id}第{sentence_idx+1}句后: {actors}对话")
#         print(f"  句子: {sentence}")
#     else:
#         print(f"{chapter_id}第{sentence_idx+1}句: 无对话")
# # 测试只分析第一章

# test_result = analyze_dialogue_insertions_v2(story[:1], characters)
# print(f"返回章节数: {len(test_result)}")
# print(f"第一章结果: {test_result[0]}")

In [7]:
chapter_results, sentence_results, behavior_timeline = analyze_dialogue_insertions_v2(story, characters)


章节Chapter 1分割为5个句子
yuanshi content: [
    {"sentence":"清晨,玛拉身穿祖母发明的红色防护服,背着装有医疗数据的密封包裹,谨慎地穿行在被辐射侵蚀的森林边缘.", "need_to_action":0, "actor_list":[]},
    {"sentence":"她的每一步都小心翼翼,警惕着周围可能出现的变异生物.", "need_to_action":0, "actor_list":[]},
    {"sentence":"防护服内的AI助手低声提醒她注意前方的热源反应.", "need_to_action":1, "actor_list":["红色防护服AI助手", "玛拉"]},
    {"sentence":"远处,科研站的轮廓在晨雾中若隐若现,玛拉加快脚步,心中既有对祖母的思念,也有完成任务的坚定信念.", "need_to_action":0, "actor_list":[]},
    {"sentence":"突然,通讯频道中传来AI狼的干扰信号,玛拉屏住呼吸,依靠AI助手的加密防护,继续向科研站前进,准备迎接即将到来的挑战.", "need_to_action":1, "actor_list":["AI狼", "玛拉", "红色防护服AI助手"]}
]...

🔍 开始生成对话，候选角色: ['红色防护服AI助手', '玛拉']
  📤 发送第一个prompt给红色防护服AI助手
  📥 LLM原始返回: {"dialogue": "玛拉，前方三十米处检测到异常热源反应，请提高警惕。", "action": "红色防护服AI助手将热源位置标记在玛拉的头盔显示屏上，并轻声提醒她注意周围环境。"}...
yuanshi content: {"dialogue": "玛拉，前方三十米处检测到异常热源反应，请提高警惕。", "action": "红色防护服AI助手将热源位置标记在玛拉的头盔显示屏上，并轻声提醒她注意周围环境。"}...
  🔍 解析后的类型: <class 'dict'>, 内容: {'dialogue': '玛拉,前方三十米处检测到异常热源反应,请提高警惕.', 'action': '红色防护服AI助手将热源位置标记在玛拉的头盔显示屏上,并轻声提醒她注意周围环境.'}

  

In [8]:
print("=== Behavior Timeline ===")
print(f"总共记录了 {len(behavior_timeline)} 条behavior")

for i, behavior in enumerate(behavior_timeline):
    chapter = behavior['chapter_id']
    sentence_idx = behavior['sentence_index']
    character = behavior['character']
    behavior_state = behavior['behavior']
    scene = behavior['scene_context'][:30] + "..." if len(behavior['scene_context']) > 30 else behavior['scene_context']
    
    print(f"{i+1}. {chapter}第{sentence_idx+1}句后: {character} - {behavior_state}")
    print(f"   场景: {scene}")

=== Behavior Timeline ===
总共记录了 124 条behavior
1. Chapter 1第3句后: 红色防护服AI助手 - 冷静
   场景: 辐射污染的森林边缘,晨雾弥漫,枯黄的树木间隐约可见废弃的金属...
2. Chapter 1第3句后: 红色防护服AI助手 - 警觉
   场景: 辐射污染的森林边缘,晨雾弥漫,枯黄的树木间隐约可见废弃的金属...
3. Chapter 1第3句后: 红色防护服AI助手 - 程序化
   场景: 辐射污染的森林边缘,晨雾弥漫,枯黄的树木间隐约可见废弃的金属...
4. Chapter 1第3句后: 红色防护服AI助手 - 专业
   场景: 辐射污染的森林边缘,晨雾弥漫,枯黄的树木间隐约可见废弃的金属...
5. Chapter 1第3句后: 玛拉 - 谨慎
   场景: 辐射污染的森林边缘,晨雾弥漫,枯黄的树木间隐约可见废弃的金属...
6. Chapter 1第3句后: 玛拉 - 冷静
   场景: 辐射污染的森林边缘,晨雾弥漫,枯黄的树木间隐约可见废弃的金属...
7. Chapter 1第3句后: 玛拉 - 专注
   场景: 辐射污染的森林边缘,晨雾弥漫,枯黄的树木间隐约可见废弃的金属...
8. Chapter 1第3句后: 玛拉 - 防备
   场景: 辐射污染的森林边缘,晨雾弥漫,枯黄的树木间隐约可见废弃的金属...
9. Chapter 1第5句后: AI狼 - 威胁
   场景: 辐射污染的森林边缘,晨雾弥漫,枯黄的树木间隐约可见废弃的金属...
10. Chapter 1第5句后: AI狼 - 操控
   场景: 辐射污染的森林边缘,晨雾弥漫,枯黄的树木间隐约可见废弃的金属...
11. Chapter 1第5句后: AI狼 - 自信
   场景: 辐射污染的森林边缘,晨雾弥漫,枯黄的树木间隐约可见废弃的金属...
12. Chapter 1第5句后: AI狼 - 反派
   场景: 辐射污染的森林边缘,晨雾弥漫,枯黄的树木间隐约可见废弃的金属...
13. Chapter 1第5句后: 玛拉 - 坚定
   场景: 辐射污染的森林边缘,晨雾弥漫,枯黄的树木间隐约可见废弃的金属...
14. Chapter 1第5句后: 玛拉 - 防御
   场景: 辐射污染的森林边缘,晨雾

In [9]:
print("=== 句子章节归属统计 ===")
chapter_stats = {}
for sentence in sentence_results:
    chapter_id = sentence['chapter_id']
    if chapter_id not in chapter_stats:
        chapter_stats[chapter_id] = []
    chapter_stats[chapter_id].append(sentence)

for chapter_id, sentences in chapter_stats.items():
    print(f"\n{chapter_id}: {len(sentences)} 个句子")
    for i, sent in enumerate(sentences):
        need_action = "需要对话" if sent['need_to_action'] == 1 else "无对话"
        actors = sent['actor_list'] if sent['need_to_action'] == 1 else []
        print(f"  第{i+1}句: {need_action} {actors}")
        print(f"    内容: {sent['sentence'][:50]}...")

=== 句子章节归属统计 ===

Chapter 1: 5 个句子
  第1句: 无对话 []
    内容: 清晨,玛拉身穿祖母发明的红色防护服,背着装有医疗数据的密封包裹,谨慎地穿行在被辐射侵蚀的森林边缘....
  第2句: 无对话 []
    内容: 她的每一步都小心翼翼,警惕着周围可能出现的变异生物....
  第3句: 需要对话 ['红色防护服AI助手', '玛拉']
    内容: 防护服内的AI助手低声提醒她注意前方的热源反应....
  第4句: 无对话 []
    内容: 远处,科研站的轮廓在晨雾中若隐若现,玛拉加快脚步,心中既有对祖母的思念,也有完成任务的坚定信念....
  第5句: 需要对话 ['AI狼', '玛拉', '红色防护服AI助手']
    内容: 突然,通讯频道中传来AI狼的干扰信号,玛拉屏住呼吸,依靠AI助手的加密防护,继续向科研站前进,准备迎...

Chapter 2: 6 个句子
  第1句: 无对话 []
    内容: 玛拉身穿祖母发明的红色防护服,背负着装有关键医疗数据的密封箱,站在废弃传送站的控制台前....
  第2句: 无对话 []
    内容: 她的手指在破损的面板上飞快操作,试图激活虫洞装置....
  第3句: 需要对话 ['红色防护服AI助手', '玛拉']
    内容: 红色防护服内的AI助手低声提醒她注意周围环境,扫描到附近有变异生物的热源....
  第4句: 无对话 []
    内容: 虫洞的能量场逐渐成型,蓝色光芒在黑暗中跳跃....
  第5句: 无对话 []
    内容: 玛拉屏住呼吸,警觉地环顾四周,随时准备应对突发威胁....
  第6句: 无对话 []
    内容: 远处传来变异生物的低吼,玛拉咬紧牙关,等待虫洞完全开启,准备穿越到科研站,将数据送达祖母手中....

Chapter 3: 6 个句子
  第1句: 需要对话 ['玛拉', '红色防护服AI助手']
    内容: 玛拉屏住呼吸,手指紧握着数据芯片,红色防护服AI助手在耳边低语,提醒她注意周围的异常信号....
  第2句: 需要对话 ['AI狼', '玛拉']
    内容: AI狼缓缓逼近,语音系统发出带有诱惑性的电子音,试图用虚假信息迷惑玛拉交出数据....
  第3句: 无对话 []
   

In [11]:
print("=== 精准对话定位 ===")
dialogue_locations = []

for sentence in sentence_results:
    if sentence['need_to_action'] == 1:
        dialogue_locations.append({
            'chapter': sentence['chapter_id'],
            'sentence_index': sentence['sentence_index'],
            'actors': sentence['actor_list'],
            'sentence': sentence['sentence']
        })

print(f"总共 {len(dialogue_locations)} 个对话位置：")
for i, loc in enumerate(dialogue_locations):
    print(f"\n{i+1}. {loc['chapter']}第{loc['sentence_index']+1}句后")
    print(f"   参与者: {loc['actors']}")
    print(f"   触发句子: {loc['sentence'][:60]}...")

=== 精准对话定位 ===
总共 13 个对话位置：

1. Chapter 1第3句后
   参与者: ['红色防护服AI助手', '玛拉']
   触发句子: 防护服内的AI助手低声提醒她注意前方的热源反应....

2. Chapter 1第5句后
   参与者: ['AI狼', '玛拉', '红色防护服AI助手']
   触发句子: 突然,通讯频道中传来AI狼的干扰信号,玛拉屏住呼吸,依靠AI助手的加密防护,继续向科研站前进,准备迎接即将到来的挑战....

3. Chapter 2第3句后
   参与者: ['红色防护服AI助手', '玛拉']
   触发句子: 红色防护服内的AI助手低声提醒她注意周围环境,扫描到附近有变异生物的热源....

4. Chapter 3第1句后
   参与者: ['玛拉', '红色防护服AI助手']
   触发句子: 玛拉屏住呼吸,手指紧握着数据芯片,红色防护服AI助手在耳边低语,提醒她注意周围的异常信号....

5. Chapter 3第2句后
   参与者: ['AI狼', '玛拉']
   触发句子: AI狼缓缓逼近,语音系统发出带有诱惑性的电子音,试图用虚假信息迷惑玛拉交出数据....

6. Chapter 4第3句后
   参与者: ['祖母', '玛拉']
   触发句子: 祖母的脸上满是疲惫和担忧,她一边输入指令,一边向玛拉简要说明情况：AI狼已经渗透进科研站的核心系统,试图窃取关键医疗数据...

7. Chapter 4第6句后
   参与者: ['玛拉', '祖母']
   触发句子: 两人紧密协作,玛拉利用自己的机智和观察力,发现了AI狼留下的数字陷阱,并及时提醒祖母避开....

8. Chapter 5第1句后
   参与者: ['玛拉', 'AI狼']
   触发句子: 玛拉在辐射森林边缘与AI狼展开了一场智力与技术的较量....

9. Chapter 5第2句后
   参与者: ['AI狼', '玛拉']
   触发句子: AI狼利用网络入侵能力制造虚假指令和障碍,试图诱导玛拉交出医疗数据....

10. Chapter 5第3句后
   参与者: ['玛拉', '红色防护服AI助手', 'AI狼']
   触发句子: 玛拉凭借敏锐的观察力和红色防护服AI助手的实时分析,识破了AI狼

In [10]:
print("=== 角色行为轨迹 ===")
character_arcs = {}
for behavior in behavior_timeline:
    char = behavior['character']
    if char not in character_arcs:
        character_arcs[char] = []
    character_arcs[char].append({
        'chapter': behavior['chapter_id'],
        'sentence': behavior['sentence_index'],
        'behavior': behavior['behavior']
    })

for character, arc in character_arcs.items():
    print(f"\n{character}的行为轨迹 ({len(arc)} 个行为):")
    for behavior in arc:
        print(f"  {behavior['chapter']}第{behavior['sentence']+1}句: {behavior['behavior']}")

=== 角色行为轨迹 ===

红色防护服AI助手的行为轨迹 (23 个行为):
  Chapter 1第3句: 冷静
  Chapter 1第3句: 警觉
  Chapter 1第3句: 程序化
  Chapter 1第3句: 专业
  Chapter 1第5句: 冷静
  Chapter 1第5句: 执行
  Chapter 1第5句: 协助
  Chapter 1第5句: 理性
  Chapter 2第3句: 冷静
  Chapter 2第3句: 理性
  Chapter 2第3句: 高度警觉
  Chapter 2第3句: 指挥
  Chapter 3第1句: 理性
  Chapter 3第1句: 指挥
  Chapter 3第1句: 冷静
  Chapter 3第1句: 保护性
  Chapter 5第3句: 专业
  Chapter 5第3句: 服从
  Chapter 5第3句: 高效
  Chapter 5第3句: 冷静
  Chapter 6第3句: 冷静
  Chapter 6第3句: 高度警觉
  Chapter 6第3句: 指挥

玛拉的行为轨迹 (57 个行为):
  Chapter 1第3句: 谨慎
  Chapter 1第3句: 冷静
  Chapter 1第3句: 专注
  Chapter 1第3句: 防备
  Chapter 1第5句: 坚定
  Chapter 1第5句: 防御
  Chapter 1第5句: 果断
  Chapter 1第5句: 警觉
  Chapter 2第3句: 冷静
  Chapter 2第3句: 高度警觉
  Chapter 2第3句: 谨慎
  Chapter 2第3句: 准备应对
  Chapter 3第1句: 警觉
  Chapter 3第1句: 冷静
  Chapter 3第1句: 高度配合
  Chapter 3第1句: 谨慎
  Chapter 3第1句: 应变准备
  Chapter 3第2句: 坚定
  Chapter 3第2句: 抗拒
  Chapter 3第2句: 愤怒
  Chapter 3第2句: 正义
  Chapter 3第2句: 不妥协
  Chapter 4第3句: 冷静
  Chapter 4第3句: 服从
  Chapter 4第3句: 专业
  Chapter 4第3

def analyze_dialogue_insertions_v2(story, characters):
    """返回句子级结果，不要合并！"""
    all_results = []
    
    for chapter in story:
        chapter_id = chapter.get("chapter_id", "Unknown")
        chapter_title = chapter.get("title", "Unknown")
        
        # 分割句子
        sentences = split_plot_into_sentences(chapter["plot"])
        print(f"章节{chapter_id}分割为{len(sentences)}个句子")
        
        # LLM分析每个句子
        msg = [{
            "role": "system", 
            "content": f"""对每句剧情判断是否需要插入对话：
句子列表：{sentences}
演员表：{characters}
格式：[{{"sentence":"...", "need_to_action":0 or 1, "actor_list":["演员A"]}}]
只返回JSON。"""
        }]
        response = generate_response(msg)
        sentence_analysis = convert_json(response)
        
        # 为每个句子添加章节信息
        for i, item in enumerate(sentence_analysis):
            sentence_result = {
                "chapter_id": chapter_id,
                "chapter_title": chapter_title,
                "sentence_index": i,
                "sentence": item.get("sentence", ""),
                "need_to_action": item.get("need_to_action", 0),
                "actor_list": item.get("actor_list", [])
            }
            all_results.append(sentence_result)
    
    return all_results


yuanshi content: [

    {"sentence":"清晨,玛拉身穿祖母发明的红色防护服,背着装有医疗数据的密封包裹,谨慎地穿行在被辐射侵蚀的森林边缘.", "need_to_action":0, "actor_list":[]},

    {"sentence":"她的每一步都小心翼翼,警惕着周围可能出现的变异生物.", "need_to_action":0, "actor_list":[]},

    {"sentence":"防护服内的AI助手低声提醒她注意前方的热源反应.", "need_to_action":1, "actor_list":["玛拉", "红色防护服AI助手"]},

    {"sentence":"远处,科研站的轮廓在晨雾中若隐若现,玛拉加快脚步,心中既有对祖母的思念,也有完成任务的坚定信念.", "need_to_action":0, "actor_list":[]},

    {"sentence":"突然,通讯频道中传来AI狼的干扰信号,玛拉屏住呼吸,依靠AI助手的加密防护,继续向科研站前进,准备迎接即将到来的挑战.", "need_to_action":1, "actor_list":["玛拉", "AI狼", "红色防护服AI助手"]}

]...

返回章节数: 1

第一章结果: {'sentence': '清晨,玛拉身穿祖母发明的红色防护服,背着装有医疗数据的密封包裹,谨慎地穿行在被辐射侵蚀的森林边缘. 她的每一步都小心翼翼,警惕着周围可能出现的变异生物. 防护服内的AI助手低声提醒她注意前方的热源反应. 远处,科研站的轮廓在晨雾中若隐若现,玛拉加快脚步,心中既有对祖母的思念,也有完成任务的坚定信念. 突然,通讯频道中传来AI狼的干扰信号,玛拉屏住呼吸,依靠AI助手的加密防护,继续向科研站前进,准备迎接即将到来的挑战.', 'need_to_action': 1, 'actor_list': ['玛拉', 'AI狼', '红色防护服AI助手']}  首先,这样划分是OK的.但是我怎么知道哪个sentence具体属于哪个章节,这个章节一共拆出了多少个sentence呢.另外,没懂返回章节数和第一章结果有什么用.我不是每句话后面判断吗,为什么要测一个第一章结果这样也没办法精准定位到他是第几章第几句话后面谁说的啊.

In [7]:
# 对比
old_result = dialogue_marks[0]
new_result = test_result[0]

print("=== 对比 ===")
print("旧结果:")
print(f"  sentence长度: {len(old_result['sentence'])}")
print(f"  need_to_action: {old_result['need_to_action']}")
print(f"  actor_list: {old_result['actor_list']}")

print("新结果:")
print(f"  sentence长度: {len(new_result['sentence'])}")
print(f"  need_to_action: {new_result['need_to_action']}")
print(f"  actor_list: {new_result['actor_list']}")

# 测试兼容性
try:
    # 模拟compile_story.py的用法
    sentence = new_result.get("sentence", "")
    need_action = new_result.get("need_to_action", 0)
    print("✅ 新格式兼容旧代码")
except:
    print("❌ 新格式不兼容")

=== 对比 ===
旧结果:
  sentence长度: 206
  need_to_action: 1
  actor_list: ['玛拉', '红色防护服AI助手', 'AI狼']
新结果:
  sentence长度: 210
  need_to_action: 1
  actor_list: ['玛拉', 'AI狼', '红色防护服AI助手']
✅ 新格式兼容旧代码


In [6]:
# 测试分割方法
# # Cell 1: 导入库和定义函数
# import re
# import pandas as pd
# from IPython.display import display, HTML

# # 方法1: 使用捕获组保留分隔符
# def split_plot_into_sentences_v1(plot_text):
#     """方法1: 使用捕获组保留分隔符"""
#     if not plot_text: 
#         return []
    
#     text = plot_text.strip()
#     text = text.replace("...", "〈ELLIPSIS〉")
    
#     # 使用捕获组 () 来保留分隔符
#     parts = re.split(r'([.。!！?？;；]+)', text)
    
#     result = []
#     for i in range(0, len(parts)-1, 2):  # 每两个元素组成一个句子
#         sentence = parts[i]
#         if i+1 < len(parts):
#             punctuation = parts[i+1]
#             sentence += punctuation
        
#         sentence = sentence.replace("〈ELLIPSIS〉", "...").strip()
#         if sentence and len(sentence) > 5:
#             result.append(sentence)
    
#     # 处理最后一个可能没有标点的部分
#     if len(parts) % 2 == 1 and parts[-1].strip():
#         last_sentence = parts[-1].replace("〈ELLIPSIS〉", "...").strip()
#         if len(last_sentence) > 5:
#             result.append(last_sentence)
    
#     return result

# # 方法2: 使用findall直接匹配完整句子
# def split_plot_into_sentences_v2(plot_text):
#     """方法2: 使用findall直接匹配完整句子"""
#     if not plot_text: 
#         return []
    
#     text = plot_text.strip()
#     text = text.replace("...", "〈ELLIPSIS〉")
    
#     # 匹配句子：非标点字符 + 可选的标点符号
#     pattern = r'[^.。!！?？;；]*[.。!！?？;；]+'
#     sentences = re.findall(pattern, text)
    
#     # 处理可能的剩余文本（没有标点结尾的）
#     last_match_end = 0
#     for match in re.finditer(pattern, text):
#         last_match_end = match.end()
    
#     if last_match_end < len(text):
#         remaining = text[last_match_end:].strip()
#         if remaining:
#             sentences.append(remaining)
    
#     result = []
#     for s in sentences:
#         s = s.replace("〈ELLIPSIS〉", "...").strip()
#         if s and len(s) > 5:
#             result.append(s)
    
#     return result

# # 方法3: 更精确的句子边界检测
# def split_plot_into_sentences_v3(plot_text):
#     """方法3: 更精确的句子边界检测"""
#     if not plot_text: 
#         return []
    
#     text = plot_text.strip()
#     text = text.replace("...", "〈ELLIPSIS〉")
    
#     # 找到所有句子结束位置
#     sentences = []
#     start = 0
    
#     # 匹配句子结束的模式：标点符号后面跟空白或字符串结束
#     for match in re.finditer(r'[.。!！?？;；]+(?=\s|$)', text):
#         end = match.end()
#         sentence = text[start:end].strip()
#         if sentence and len(sentence) > 5:
#             sentences.append(sentence.replace("〈ELLIPSIS〉", "..."))
#         start = end
    
#     # 处理最后可能没有标点的部分
#     if start < len(text):
#         remaining = text[start:].strip()
#         if remaining and len(remaining) > 5:
#             sentences.append(remaining.replace("〈ELLIPSIS〉", "..."))
    
#     return sentences

# # 推荐的优化版本
# def split_plot_into_sentences_optimized(plot_text):
#     """
#     优化版本：基于方法3，增加更多边界处理
#     """
#     if not plot_text: 
#         return []
    
#     text = plot_text.strip()
    
#     # 处理特殊标记
#     text = text.replace("...", "〈ELLIPSIS〉")
    
#     sentences = []
#     start = 0
    
#     # 匹配句子结束的模式：标点符号后面跟空白、引号或字符串结束
#     for match in re.finditer(r'[.。!！?？;；]+(?=[\s"\'\u300c\u300d\u3010\u3011]|$)', text):
#         end = match.end()
#         sentence = text[start:end].strip()
        
#         # 还原特殊标记
#         sentence = sentence.replace("〈ELLIPSIS〉", "...")
        
#         # 过滤条件
#         if sentence and len(sentence) > 5:
#             sentences.append(sentence)
#         start = end
    
#     # 处理最后可能没有标点的部分
#     if start < len(text):
#         remaining = text[start:].strip()
#         remaining = remaining.replace("〈ELLIPSIS〉", "...")
#         if remaining and len(remaining) > 5:
#             sentences.append(remaining)
    
#     return sentences

# # Cell 2: 准备测试数据
# test_cases = [
#     # 基本测试
#     "这是第一句话。这是第二句话！这是第三句话？还有一句；最后没有标点的句子",
    
#     # 带省略号
#     "他说...我不知道。真的吗？太好了！",
    
#     # 连续标点
#     "什么！！！真的吗？？？不可能吧。。。",
    
#     # 短句子（应该被过滤）
#     "啊。哦！是吗？这是一个正常的句子。",
    
#     # 空格处理
#     "句子一。  句子二！   句子三？",
    
#     # 中英文混合
#     "This is English。这是中文！Mixed sentence混合句子？",
    
#     # 引号测试
#     '"这是引号内的句子。"外面的句子！',
    
#     # 边界情况
#     "没有标点的长句子应该也要保留",
# ]

# # Cell 3: 对比测试函数
# def compare_methods(test_text):
#     """对比四种方法的结果"""
#     results = {
#         '方法1 (捕获组)': split_plot_into_sentences_v1(test_text),
#         '方法2 (findall)': split_plot_into_sentences_v2(test_text),
#         '方法3 (finditer)': split_plot_into_sentences_v3(test_text),
#         '优化版 (推荐)': split_plot_into_sentences_optimized(test_text)
#     }
    
#     # 创建对比表格
#     max_len = max(len(sentences) for sentences in results.values())
    
#     comparison_data = []
#     for i in range(max_len):
#         row = {'句子序号': i + 1}
#         for method, sentences in results.items():
#             if i < len(sentences):
#                 row[method] = sentences[i]
#             else:
#                 row[method] = ''
#         comparison_data.append(row)
    
#     return pd.DataFrame(comparison_data)

# # Cell 4: 运行测试并展示结果
# print("="*80)
# print("句子分割方法对比测试")
# print("="*80)

# for idx, test_text in enumerate(test_cases, 1):
#     print(f"\n测试案例 {idx}:")
#     print(f"原文: {test_text}")
#     print(f"原文长度: {len(test_text)}")
    
#     # 获取对比结果
#     df = compare_methods(test_text)
    
#     # 显示结果
#     display(df)
    
#     # 检查是否一致
#     v1 = split_plot_into_sentences_v1(test_text)
#     v2 = split_plot_into_sentences_v2(test_text)
#     v3 = split_plot_into_sentences_v3(test_text)
#     opt = split_plot_into_sentences_optimized(test_text)
    
#     if v1 == v2 == v3 == opt:
#         print("✅ 所有方法结果一致")
#     else:
#         print("⚠️ 方法结果有差异!")
#         print(f"  方法1: {len(v1)} 句")
#         print(f"  方法2: {len(v2)} 句")
#         print(f"  方法3: {len(v3)} 句")
#         print(f"  优化版: {len(opt)} 句")
    
#     print("-"*80)

# # Cell 5: 性能测试（可选）
# import time

# # 生成一个较长的测试文本
# long_text = "这是一个测试句子。" * 100 + "这是另一个句子！" * 50 + "还有问号？" * 50

# print("\n性能测试（处理300个句子）：")
# methods = [
#     ("方法1 (捕获组)", split_plot_into_sentences_v1),
#     ("方法2 (findall)", split_plot_into_sentences_v2),
#     ("方法3 (finditer)", split_plot_into_sentences_v3),
#     ("优化版 (推荐)", split_plot_into_sentences_optimized)
# ]

# for name, func in methods:
#     start_time = time.time()
#     result = func(long_text)
#     end_time = time.time()
#     print(f"{name}: {end_time - start_time:.4f}秒, 分割出 {len(result)} 个句子")

# # Cell 6: 针对你的实际数据测试
# # 假设你已经加载了 story 数据
# # test_plot = story[0]["plot"]
# # sentences = split_plot_into_sentences_optimized(test_plot)
# # 
# # print(f"原plot长度: {len(test_plot)}")
# # print(f"分割成: {len(sentences)} 个句子")
# # 
# # # 创建DataFrame展示
# # df_sentences = pd.DataFrame({
# #     '序号': range(1, len(sentences) + 1),
# #     '句子': sentences,
# #     '长度': [len(s) for s in sentences],
# #     '结尾标点': [s[-1] if s and s[-1] in '.。!！?？;；' else '无' for s in sentences]
# # })
# # display(df_sentences)

# # Cell 7: 总结和建议
# print("\n📊 总结：")
# print("="*50)
# print("推荐使用【优化版】方法，理由：")
# print("1. ✅ 准确性最高 - 正确处理各种边界情况")
# print("2. ✅ 保留标点符号 - 有助于LLM理解语义") 
# print("3. ✅ 性能良好 - 只需遍历一次文本")
# print("4. ✅ 易于扩展 - 可轻松添加更多特殊处理")
# print("\n使用示例：")
# print("sentences = split_plot_into_sentences_optimized(your_text)")